# Exercise 7.1. EKF SLAM

## Theoretical background

Simultaneous Localization And Mapping or SLAM is one of the most fundamental problems in robotics
It arises when our robot doesn't have access to a map of the environment nor the pose where it is located.
It is a more difficult problem than the two separate problems we did in the previous units.

There are two variants of the SLAM problem:

- **Online SLAM** (the one we will implement): Which only estimates the latest pose. $p(x_{k}, m_{1:L} | z_{1:k}, u_{1:k})$
- **Full SLAM**: Estimates the whole path traversed at each step. $p(x_{1:k}, m_{1:L} | z_{1:k}, u_{1:k})$

The EKF algorithm was originally one of the most influential approach to the SLAM problem.

This approach is similar to the one we took to the problems of Localization and Mapping.



The exercise’s appendix includes a code implementing a SLAM algorithm based on the
Extended Kalman Filter, but it’s incomplete at some points.

**Example**

The following image is an exaple of a correct execution of the demo:


<figure style="text-align:center">
  <img src="images/fig7-1-1.png" width="400" alt="">
  <figcaption>
      Fig. 1: Execution of the EKF algorithmn for SLAM. <br/>
      it shows 3 poses: true (blue), expected (red) and estimated (green + confidence ellipse); <br/>
      true landmarks (big multicolored squares), and 
      their final <br/> estimations (green squares + red confidence ellipse)
  </figcaption>
</figure>

In [ ]:
%matplotlib notebook

import time
import math

import numpy as np
from numpy import random
from scipy import linalg
import matplotlib
#matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
import pandas as pd

from utils.tcomp import tcomp
from utils.Jacobians import J1, J2
from utils.DrawRobot import DrawRobot
from utils.PlotEllipse import PlotEllipse
from utils.AngleWrap import AngleWrap
from utils.unit7.FOV import FOVSensor
from utils.unit7.Jacobians import GetNewFeatureJacs, GetObsJacs
from utils.unit7.MapCanvas import MapCanvas
from utils.unit7.Robot import EFKSlamRobot

To simplify a bit the code we will use the following classes:

- [FOVSensor](/edit/utils/unit7/FOV.py): Take a look at the parameters it contains and its functions.
- [EFKSlamRobot](/edit/utils/unit7/Robot.py): We'll only use its parameters and step function.

Complete the missing parts of the code and then answer the following questions:

- 1. What represents PPred and why is it build in that way in the EFK function? Which are its dimensions? and those of the matrices used to build it? (`PPredvv`, `PPredvm` and `PPredmm`)

- 2. Build the state Jacobian jH used in the Kalman filter during the update step when a previously perceived landmark is seen again. Employ the output of the `GetObsJacs` function. Analyze both its size and its content throughout the SLAM simulation.

In [ ]:
def EFKSlam(robot: EFKSlamRobot, sensor: FOVSensor, z, iFeature, u):
    """ Implementation of the EFK algorithm for SLAM.
    
        It does not return anything.
        Just updates the state attributes in robot(causing side effects only in robot). 
    
        Args:
            robot
            sensor
            z: observation made in this loop
            iFeature: Index of the landmark observed in the world map and in robot.MappedFeatures.
                It serves to chech whether it is in the state and if so, where is located.
            u: Movement command received in this loop. 
                It serves us to predict the future pose in the state(xVehicle).
                At the time this function is called, robot.pose and robot.true_pose have been updated already.
    
    """
    # Useful vbles
    xVehicle = robot.xEst[0:3]
    xMap = robot.xEst[3:]

    #
    # Prediction step
    #
    xVehiclePred = tcomp(xVehicle,u)

    j1 = J1(xVehicle,u)
    j2 = J2(xVehicle,u)

    PPredvv = j1 @ robot.PEst[0:3,0:3] @ j1.T + j2@ robot.cov_move @ j2.T
    PPredvm = j1@robot.PEst[0:3,3:]
    PPredmm = robot.PEst[3:,3:]

    xPred = np.vstack([xVehiclePred,xMap])
    PPred = np.vstack([
        np.hstack([PPredvv, PPredvm]),
        np.hstack([PPredvm.T, PPredmm])
    ])

    #
    # Update step
    #
    if z.shape[1] > 0:
        #have we seen this feature before?
        if robot.MappedFeatures[iFeature,0] >=0:

            #predict observation: find out where it is in state vector
            FeatureIndex = robot.MappedFeatures[iFeature,:]
            xFeature = xPred[FeatureIndex[0]:FeatureIndex[1]]

            zPred = sensor.observe(xVehiclePred,xFeature, noisy=False)

            # get observation Jacobians
            jHxv,jHxf = GetObsJacs(xVehicle,xFeature)
            # Fill in state jacobian

            #
            # TODO Point 2, Build jH from JHxv and jHxf
            #
            
            raise NotImplementedError
            jH = None

            # Do Kalman update:
            Innov = z-zPred
            Innov[1] = AngleWrap(Innov[1])

            S = jH@PPred@jH.T + sensor.cov_sensor
            W = PPred@jH.T@linalg.inv(S)
            
            robot.xEst = xPred+ W@Innov
            robot.PEst = PPred-W@S@W.T

            #ensure P remains symmetric
            robot.PEst = 0.5*(robot.PEst+robot.PEst.T)
        else:
            # this is a new feature add it to the map....
            nStates = len(robot.xEst)

            xFeature = (
                xVehiclePred[0:2] + 
                np.vstack([
                    z[0]*np.cos(z[1]+xVehiclePred[2]),
                    z[0]*np.sin(z[1]+xVehiclePred[2])
                ])
            )
            robot.xEst = np.vstack([xPred,xFeature]) #augmenting state vector
            jGxv, jGz = GetNewFeatureJacs(xVehicle,z)
            
            M = np.vstack([
                np.hstack([np.eye(nStates), np.zeros((nStates,2))]),# note we don't use jacobian w.r.t vehicle
                np.hstack([jGxv, np.zeros((2,nStates-3)), jGz]),
            ])
            robot.PEst = M@linalg.block_diag(robot.PEst,sensor.cov_sensor)@M.T

            #remember this feature as being mapped we store its ID and position in the state vector
            robot.MappedFeatures[iFeature,:] = [len(robot.xEst)-2, len(robot.xEst)]
    else:
        robot.xEst = xPred
        robot.PEst = PPred

We will use the following demo to test our robot.

In [ ]:
def main(robot,
         sensor,
         nFeatures=10,
         MapSize=200,
         DrawEveryNFrames=5,
         nSteps = 195,
         turning = 50,
         mode='one_landmark_in_fov',
         NONSTOP=True,
         LOG=False):
    
    
    %matplotlib notebook
    if not NONSTOP:
        matplotlib.use('TkAgg')


    seed = 100
    np.random.seed(seed)

    logger = Logger(nFeatures, nSteps);
    
    # Map configuration
    Map = MapSize*random.rand(2, nFeatures) - MapSize/2

    # Matplotlib setup
    canvas = MapCanvas(Map, MapSize, nFeatures, robot, sensor, NONSTOP)
    canvas.initialFrame(robot, Map, sensor)

    u = np.vstack([3.0, 0.0, 0.0])

    for k in range(1, nSteps):
        
        # Move the robot with a control action u
        u[2] = 0.0
        if k%turning == 0:
             u[2]=np.pi/2

        robot.step(u)

        # Get new observation/s
        if mode == 'one_landmark_in_fov' :
            # Get a random observations within the fov of the sensor
            z, iFeature = sensor.random_observation(robot.true_pose, Map, fov=True)
        elif mode == 'landmarks_in_fov':
            # OPTIONAL Point 4
            raise NotImplementedError

        EFKSlam(robot, sensor, z, iFeature, u)

        # Point 3, Robot pose and features localization errors and determinants
        if logger is not None:
            logger.log(k, robot, Map) 

        # Drawings
        if k % DrawEveryNFrames == 0:
            canvas.drawFrame(robot, sensor, Map, iFeature)
            
    # Draw the final estimated positions and uncertainties of the features
    canvas.drawFinal(robot)

    if logger is not None:
        %matplotlib inline
        logger.draw(canvas.colors)

- 3. Store in each iteration the determinant of the matrices of covariance of the robot pose and the localization of each feature and plot them. Do the same with the error of the localization of the pose and the features. Use the variables `PFeatDetStore`, `FeatErrStore`, `PXErrStore` and `XErrStore`.

  Once this has been completed, the results will look similar to the following figutes:
  
  <table>
    <tr>
        <td><img src="images/fig7-1-2.png" width="300"></td>
        <td><img src="images/fig7-1-3.png" width="300"></td>
    </tr>
    <tr>
        <td><img src="images/fig7-1-4.png" width="300"></td>
        <td><img src="images/fig7-1-5.png" width="300"></td>
    <tr>
  </table>
  
  Just as in the previous practice you'll have to complete a Logger class that is used along the demo in `main()`.

In [ ]:
class Logger():
    def __init__(self, nFeatures, nSteps):
        # Storage:
        self.PFeatDetStore = np.full((nFeatures,nSteps),np.Inf)
        self.FeatErrStore = np.full((nFeatures,nSteps),np.Inf)
        self.PXErrStore = np.full((nSteps,1), 0)
        self.XErrStore = np.full((2,nSteps), 0) # error in position and angle
        
    def log(self, k, robot, Map):
        
        # TODO. Store relevant info
        raise NotImplementedError
        
    def draw(self, colors):
        nSteps = self.PFeatDetStore.shape[1]
        nFeatures = self.PFeatDetStore.shape[0]
        
        plt.figure(2) #hold on
        plt.title('Errors in robot localization')
        plt.plot(self.XErrStore[0,:],'b',label="Error in position")
        plt.plot(self.XErrStore[1,:],'r',label="Error in orientation")
        #plt.legend('Error in position','Error in orientation')
        plt.legend()

        plt.figure(3)# hold on
        plt.title('Determinant of the cov. matrix associated to the robot localization')
        xs = np.arange(nSteps)
        plt.plot(self.PXErrStore[:],'b')

        plt.figure(4)# hold on
        plt.title('Errors in features localization')

        plt.figure(5)# hold on
        plt.title('Log of the determinant of the cov. matrix associated to each feature')

        for i in range(nFeatures):
            plt.figure(5)
            h = plt.plot(np.log(self.PFeatDetStore[i,:]), color=colors[i,:])
            plt.figure(4)
            h = plt.plot(self.FeatErrStore[i,:], color=colors[i,:])

    - 3.1 - Now the program is complete. Run it a few times and describe the obtained results. Explain the meaning of each element appearing in the figure resulting from the execution of the SLAM algorithm. (See a running example in the next page).
    - 3.2 - Play with different numbers of landmarks and discuss the results.

In [ ]:
# Map configuration
n_features = 10
MapSize = 200

# Robot base characterization
SigmaX = 0.01 # Standard deviation in the x axis
SigmaY = 0.01 # Standard deviation in the y axins
SigmaTheta = 1.5*np.pi/180 # Bearing standar deviation
R = np.diag([SigmaX**2, SigmaY**2, SigmaTheta**2]) # Cov matrix

xRobot = np.vstack([-MapSize/3, -MapSize/3, 0.0])
robot = EFKSlamRobot(xRobot, R, n_features)

Sigma_r = 1.1
Sigma_theta = 5*np.pi/180
Q = np.diag([Sigma_r, Sigma_theta])**2 # Covariances for our very bad&expensive sensor (in the system <d,theta>)
fov = np.pi*2/3
max_range = 100

sensor = FOVSensor(Q, fov, max_range) 

#main(robot, sensor, nFeatures=n_features, MapSize=MapSize, NONSTOP=False)

- 4. Optional. The provided code only employs a feature per iteration. Change the code to consider all the features within the FOV of the robot during the update step of the EKF. Creating a backup is recommended before continuing.